
<div align="center">
<h2 style="text-align: center;" markdown="1">Journées Mathrice</h2>
<img alt="Mathrice" style="border-width:0" width="15%" src="images/logo_mathrice.png">
<h3 style="text-align: center;" markdown="1">Montpellier, 27-29 mars 2018</h3>
</div>                                       
<hr>
<h1 style="text-align: center;" markdown="0">Publier sur math.cnrs.fr avec GitLab Pages</h1>
<br>
<div align="center">
<a rel="diaporama" href="index.html">Diaporama</a> -
<a rel="html" href="gitlab-pages.html">Page unique</a>
</div>                                       
<hr>
<h4 style="text-align: center;" markdown="0"><em>Matthieu Boileau</em></h4>
<h4 style="text-align: center;" markdown="0"><em>IRMA, CNRS, Université de Strasbourg</em></h4>
<hr>


<div align="center">
<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Licence Creative Commons" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png"/></a>
</div>

<img alt="Settings" width="15%" src="images/stacked_wm_no_bg.png">

# GitLab Pages

À partir de projets hébergés sur <https://plmlab.math.cnrs.fr>, GitLab Pages permet de :

- publier un **site web statique**, construit par exemple avec un générateur de site statique,
- mettre en ligne des **pdf** d'articles de livres, de cours, d'énoncé d'exercices, etc., construits à partir de sources Tex, par exemple.
- mettre en ligne des **diaporamas**, comme celui-ci...



<img alt="Settings" width="10%" src="images/pages_group_avatar.png">

# Du point de vue l'utilisateur

<img alt="Avantages" width="5%" src="images/plus_fin_alpha.png">


## Permet de publier sur une URL en `math.cnrs.fr`

Pages permet de publier sur une URL qui contient un nom de projet ou d'équipe.

Pour le projet GitLab `projectname` qui appartient à
`username` ou `groupname`, l'adresse de publication sera :

```
https://[username|groupname].pages.math.cnrs.fr/[projectname]
```

> `projectname` est absent de l'URL s'il vaut `[username|groupname].pages.math.cnrs.fr`


<img alt="Avantages" width="5%" src="images/plus_fin_alpha.png">

## Permet de faire de l'édition collaborative

- Utiliser Git et GitLab pour **enregistrer et fusionner** les différentes contributions
- Utiliser GitLab Pages pour **reconstruire** le site ou le document en ligne après chaque modification sur le dépôt GitLab

<img alt="Avantages" width="5%" src="images/plus_fin_alpha.png">

## Très facile à mettre en oeuvre

En deux temps, trois mouvements :

1. On enregistre un `gitlab runner` ou on utilise un *runner* partagé
2. On ajoute un fichier `.gitlab-ci.yml` décrivant le job d'intégration continue

# Limitations et solutions

- **pas de contenu web dynamique** mais les générateurs de sites statiques font déjà beaucoup !
- une **taille maximale** de contenu publiable (100Mo par défaut) mais peut être étendu par l'admin
- **l'URL de publication** contient forcément `.pages.math.cnrs.fr` mais si l'admin l'active, on peut [ajouter un nom de domaine personnalisé](https://docs.gitlab.com/ee/user/project/pages/introduction.html#add-a-custom-domain-to-your-pages-website).

<img alt="Settings" width="10%" src="images/pages_group_avatar.png">

# Du point de vue de l'administrateur

<img alt="Avantages" width="8%" src="images/plus_fin_alpha.png">

##  Avantages

- aucune intervention nécessaire de la part de l'admin : la publication est automatiquement déclenchée par le job d'intégration continue
- on évite de gérer à la main "un site web => un serveur web (ou un virtualhost)" : Pages le fait automatiquement !
- pas de problème de sécurité lié à du contenu dynamique : l'exécution de code (php ou autre) n'est pas autorisée.

<img alt="Settings" width="8%" src="images/gear.png">

## Configuration

Quelques lignes dans le fichier `/etc/gitlab/gitlab.rb` :

```ruby
################
# gitlab pages #
################
pages_external_url 'https://pages.math.cnrs.fr'

pages_nginx['redirect_http_to_https'] = true
pages_nginx['ssl_certificate'] = "/etc/gitlab/ssl/star_pages_math_cnrs_fr.pem"
pages_nginx['ssl_certificate_key'] = "/etc/gitlab/ssl/star_pages_math_cnrs_fr.key"
```

- les 3 dernières lignes sont nécessaires pour avoir un site en https
- `star_pages_math_cnrs_fr.[pem|key]` sont les fichiers du certificat wildcard qui est nécessaire pour publier sur `*.pages.math.cnrs.fr`.£

## Suivi de l'utilisation de Pages

La version actuelle de GitLab ne fournit pas d'outil pour monitorer l'utilisation de Pages. Voici un [script](https://gist.github.com/boileaum/e28486a4a2f0e0f520fa35a410f96400) permettant de lister les contenus publiés avec Pages :

```bash
sudo gitlab-admin/gitlab_pages.py 
[sudo] password for xxx: 
Exploring default Pages path: /var/opt/gitlab/gitlab-rails/shared/pages/ 
tonus:
	https://tonus.pages.math.unistra.fr/schnaps/kirsch (9,5M)
	https://tonus.pages.math.unistra.fr/schnaps/schnaps (119M)
formation-python:
	https://formation-python.pages.math.unistra.fr (5,2M)
[...]
formation-info:
	https://formation-info.pages.math.unistra.fr/gitlab-pages (32K)
	https://formation-info.pages.math.unistra.fr/expose-git (1,3M)
	https://formation-info.pages.math.unistra.fr/atelier-gitlab --> WARNING: directory is empty
	https://formation-info.pages.math.unistra.fr/expose-git-example/methodus-inveniendi-addII.pdf (2,3M)
boileau:
	https://boileau.pages.math.unistra.fr/tp-gitlab (4,0M)
	https://boileau.pages.math.unistra.fr/website (68K)
	https://boileau.pages.math.unistra.fr/relax (4,1M)

```

<img alt="Attention" width="10%" src="images/warning_alpha.png">

## Précautions

- il est très facile passer du contenu privé (projet gitlab) à du contenu public (Pages)
- privilégier la publication d'une branche identifiée (généralement `master`)
- s'assurer que les utilisateurs adhèrent à une charte


<img alt="Play" width="20%" src="images/show.png">


# Démonstration

- **Objectif :** mettre en ligne la présentation actuelle
- Contrairement à la plupart des démos disponibles en ligne, **on part de zéro.**


## Etape 1 : création d'un projet sur `plmlab.math.cnrs.fr`



- Sur <https://plmlab.math.cnrs.fr/projects/new>, on crée le projet `gitlab-pages-demo`
- On réiniatilise le dépôt local actuel pour la démo

In [ ]:
export USERNAME=mboileau  # Remplacez par le vôtre
./reset.sh

- Dans le dépôt local existant, on ajoute le remote `demo` :

In [ ]:
git remote add demo git@plmlab.math.cnrs.fr:$USERNAME/gitlab-pages-demo.git

- On pousse le dépôt git actuel vers ce nouveau remote :

In [ ]:
git push demo

- On bascule sur une branche de dev :

In [ ]:
git checkout -b dev

## Etape 2 : identification d'un *gitlab runner*

- GitLab Pages repose sur le workflow d'intégration continue de GitLab : gitlab-ci
- Avec gitlab-ci, les jobs d'intégration continue sont assurés par les *gitlab runners*
- Pour utiliser un *runner* spécifique, cf. Annexe
- L'instance `plmlab.math.cnrs.fr` propose des ***runners*** **partagés**
- On explore les *runners disponibles* sur la page du projet dans la [page dédiée aux *runners*](https://plmlab.math.cnrs.fr/mboileau/gitlab-pages-demo/settings/ci_cd) :

```
CI / CD > Runners settings
```

- On choisit le *runner* partagé identifié par le tag `dind` pour [*Docker in Docker*]().

## Etape 3 : configuration du fichier `gitlab-ci.yml`

Avec GitLab-CI, toute l'intégration continue est décrite par un fichier unique `.gitlab-ci.yml` qui doit être présent dans la racine du projet :

In [ ]:
cat > .gitlab-ci.yml <<- EOM
image: boileaum/jupyter

pages:
  script:
    - ./build.sh
  tags:
    - dind
  only:
    - master
  artifacts:
    paths:
      - public
EOM

avec :

- `image: boileaum/jupyter` l'image [`boileaum/jupyter`](https://hub.docker.com/r/boileaum/jupyter/) du conteneur Docker chargé d'exécuter le script d'intégration continue 
- `pages:` le job gitlab-ci doit obligatoirement porter ce nom
- `build.sh` un script qui convertit ce notebook Jupyter en diaporama Reveal.js
- `public/` le répertoire dont le contenu sera publié par Pages

```yaml
  only:
    - master
```
Ce job ne sera exécuté que pour la branche `master`.

```yaml
  tags:
    - dind
```
Avec le tag `dind`, on cible le *runner* partagé.

```yaml
  artifacts:
    paths:
      - public
```

Nécessaire pour que le répertoire soit effectivement publié.

- On enregistre dans git

In [ ]:
git add .gitlab-ci.yml gitlab-pages.ipynb
git commit -m "Add .gitlab-ci.yml to trigger GitLab Pages"

## Etape 4 : on pousse vers le remote

In [ ]:
git push -u demo

> Cette action ne déclenche pas la chaîne d'intégration continue car cette dernière ne concerne que la branche `master`.

- On ouvre le lien affiché dans le retour console pour ouvrir une *merge request* sur GitLab
- On fusionne
- On surveille l'exécution du job d'intégration continue dans GitLab :
```
CI/CD > Pipelines
```
- Une fois le job CI terminé, on va dans la rubrique Pages du projet pour récupérer l'URL de publication :
```
Settings > Pages
```
Elle vaut ici <http://mboileau.pages.math.cnrs.fr/gitlab-pages-demo>

## Publication vers une adresse courte

- On aurait pu publier sur <http://mboileau.pages.math.cnrs.fr> en renommant simplement notre projet :
```
mboileau.pages.math.cnrs.fr
```
- Autre solution : créer le groupe GitLAb `demo` (par exemple) et y déplacer le projet en le renommant :
```
demo.pages.math.cnrs.fr
```
afin de le publier sur <http://demo.pages.math.cnrs.fr>.

# Conclusion

- GitLab Pages facilite la publication de contenu : sites web, documentation de code, articles, support de cours, d'exposés, etc., le tout sur une URL institutionnelle.
- Pages fait partie intégrante de la chaîne d'intégration continue de GitLab
- gitlab-ci permet de gérer la construction du contenu html à partir de sources et de bénéficier du workflow GitLab
- Combiner à générateur de site statique, GitLab Pages peut être une façon de collaborer sur un site en se passant d'un CMS.

# Ressources

- [Documentation officielle](https://docs.gitlab.com/ce/user/project/pages/) très complète
- De nombreux [exemples](https://gitlab.com/pages) pour démarrer
- Un [tutoriel vidéo](https://www.youtube.com/watch?v=TWqh9MtT4Bg) qui présente l'essentiel en 3min40.

## Annexe : enregistrement d'un *gitlab runner*

GitLab Pages repose sur le workflow d'intégration continue de GitLab : gitlab-ci.
Avec gitlab-ci, les jobs d'intégration continue sont assurés par les *gitlab runners*.

Pour plus de souplesse, on enregistre un *gitlab runner* de type `Docker` en suivant [la doc officielle](https://docs.gitlab.com/runner/commands/README.html#gitlab-runner-register).

- On récupère le token d'intégration continue du projet sur la page du projet dans 

```
Settings > CI / CD > Runners settings
```

- On positionne la variable d'environnement correspondante

In [ ]:
#export REGISTRATION_TOKEN=my-registration-token

- On enregistre le runner

In [ ]:
gitlab-runner register --non-interactive \
                       --name gl_runnner_for_$USER \
                       --url "https://plmlab.math.cnrs.fr" \
                       --executor docker \
                       --docker-image boileaum/jupyter \
                       --tag-list docker

- `--docker-image boileaum/jupyter` : le conteneur Docker chargé d'exécuter le script d'intégration continue sera basé sur l'image [`boileaum/jupyter`](https://hub.docker.com/r/boileaum/jupyter/)
- `--tag-list docker` : pour solliciter ce runner, il faudra donc que le job CI porte le tag `docker`

On vérifie que le *runner* est actif

In [ ]:
gitlab-runner list